In [2]:
!pip install -U langchain-ollama langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 40.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.4 which is incompatible.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls /content/drive/MyDrive/review_dataset

w_review_train.csv  w_review_train.parquet


In [5]:
import duckdb

con = duckdb.connect(database=':memory:', read_only=False)

In [6]:
df = con.execute("SELECT * FROM read_csv_auto('/content/drive/MyDrive/review_dataset/w_review_train.csv', parallel=false, encoding='UTF-8', ignore_errors=true)").fetchdf()

df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5


In [7]:
df.shape

(40000, 2)

In [8]:
con.execute("COPY df TO '/content/drive/MyDrive/review_dataset/w_review_train.parquet' (FORMAT PARQUET)")

In [9]:
parquet_avg_rating_direct = con.execute("SELECT AVG(column1) FROM '/content/drive/MyDrive/review_dataset/w_review_train.parquet' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [10]:
parquet_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.parquet'

In [11]:
parquet_avg_rating_direct = con.execute(f"SELECT AVG(column1) FROM '{parquet_file_path}' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [12]:
coffee_keywords = [
    'กาแฟ', 'คาเฟ่', 'coffee', 'cafe', 'เอสเปรสโซ่', 'ลาเต้',
    'มอคค่า', 'คาปูชิโน่', 'เฟรนช์เพรส', 'cold brew', 'drip',
    'espresso', 'latte', 'americano', 'macchiato', 'brew',
    'specialty', 'slow bar', 'ร้านกาแฟ', 'ร้านคาเฟ่', 'กาแฟสด',
    'เมนูกาแฟ', 'กาแฟนม', 'กาแฟดำ', 'ร้านบาริสต้า', 'barista', 'coffee shop'
]

where_clause = " OR ".join([f"column0 LIKE '%{keyword}%'" for keyword in coffee_keywords])

In [13]:
%%time
coffee_reviews_df = con.execute(f"SELECT * FROM '{parquet_file_path}' WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 4.33 s, sys: 42.7 ms, total: 4.37 s
Wall time: 3.03 s


In [14]:
csv_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.csv'

In [15]:
%%time
coffee_reviews_df_csv = con.execute(f"SELECT * FROM read_csv_auto('{csv_file_path}', parallel=false, encoding='UTF-8', ignore_errors=true) WHERE {where_clause}").fetchdf()

CPU times: user 2.05 s, sys: 60.3 ms, total: 2.11 s
Wall time: 2.18 s


In [16]:
coffee_reviews_df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
2,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
3,ร้านอาหารญี่ปุ่นร้านนี้ ใจจริงไม่อยากแนะนำเลยค...,5
4,เดือนแรกที่เค้าต่อคิวกัน 2 - 3 ชั่วโมง เราก็ว่...,5


In [17]:
coffee_reviews_df.shape

(4981, 2)

In [18]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [19]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [20]:
!ollama pull gemma3:270m

In [21]:
!ollama list

NAME           ID              SIZE      MODIFIED               
gemma3:270m    e7d36fb2c3b3    291 MB    Less than a second ago    


In [22]:
!ollama run gemma3:270m "สวัสดี"

สวัสดีค่ะ ยินดีที่ได้รู้จักนะคะ 😊




In [23]:
!curl -s http://127.0.0.1:11434/api/tags

{"models":[{"name":"gemma3:270m","model":"gemma3:270m","modified_at":"2025-11-08T16:23:18.555806527Z","size":291554930,"digest":"e7d36fb2c3b3293cfe56d55889867a064b3a2b22e98335f2e6e8a387e081d6be","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"268.10M","quantization_level":"Q8_0"}}]}

In [24]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [25]:
llm = OllamaLLM(model="gemma3:270m")

In [26]:
response = llm.invoke("สวัสดี")
print(response)

สวัสดีค่ะ ยินดีที่ได้รู้จักค่ะ 😊



In [27]:
template = """
คุณเป็นครูสอน Python
คำถาม : {question}
"""

In [28]:
prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

In [29]:
formatted_prompt = prompt.format(question="เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10")
print(formatted_prompt)


คุณเป็นครูสอน Python
คำถาม : เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10



In [30]:
chain = prompt | llm

In [31]:
response = chain.invoke({"question": "เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10"})

In [32]:
response

'```python\ndef find_sum(numbers):\n  """\n  หาผลรวมของตัวเลขในชุดที่กำหนด\n  """\n  total = 0\n  for number in numbers:\n    total += number\n  return total\n\n# ตัวอย่างการใช้งาน\nnumbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\nresult = find_sum(numbers)\nprint(result)\n```\n\n**คำอธิบายโค้ด:**\n\n1.  **`def find_sum(numbers):`**:  กำหนดชื่อ funzione `find_sum` ที่จะทำการค้นหาผลรวมของตัวเลข\n2.  **`total = 0`**:  เริ่มต้นค่า `total` เป็น 0\n3.  **`for number in numbers:`**:  วนซ้ำผ่านแต่ละตัวเลขในชุด `numbers`\n4.  **`total += number`**:  เพิ่มค่า `number` เข้าไปใน `total`  (ค่าเพิ่มเข้าไปใน `total` จะทำให้ `total`  เป็นค่าคงที่)\n5.  **`return total`**:  คืนค่า `total`  ที่ได้จากการคำนวณ\n6.  **`numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`**:  สร้างชุดข้อมูล `numbers`  เป็น `[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`\n7.  **`result = find_sum(numbers)`**:  เรียกใช้ฟังก์ชัน `find_sum`  เพื่อหาผลรวมของตัวเลขในชุด `numbers`\n8.  **`print(result)`**:  แสดงผลลัพธ์ของการค้นหาผลรวม\n'

In [33]:
from IPython.display import display, Markdown

In [34]:
display(Markdown(response))

```python
def find_sum(numbers):
  """
  หาผลรวมของตัวเลขในชุดที่กำหนด
  """
  total = 0
  for number in numbers:
    total += number
  return total

# ตัวอย่างการใช้งาน
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = find_sum(numbers)
print(result)
```

**คำอธิบายโค้ด:**

1.  **`def find_sum(numbers):`**:  กำหนดชื่อ funzione `find_sum` ที่จะทำการค้นหาผลรวมของตัวเลข
2.  **`total = 0`**:  เริ่มต้นค่า `total` เป็น 0
3.  **`for number in numbers:`**:  วนซ้ำผ่านแต่ละตัวเลขในชุด `numbers`
4.  **`total += number`**:  เพิ่มค่า `number` เข้าไปใน `total`  (ค่าเพิ่มเข้าไปใน `total` จะทำให้ `total`  เป็นค่าคงที่)
5.  **`return total`**:  คืนค่า `total`  ที่ได้จากการคำนวณ
6.  **`numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`**:  สร้างชุดข้อมูล `numbers`  เป็น `[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`
7.  **`result = find_sum(numbers)`**:  เรียกใช้ฟังก์ชัน `find_sum`  เพื่อหาผลรวมของตัวเลขในชุด `numbers`
8.  **`print(result)`**:  แสดงผลลัพธ์ของการค้นหาผลรวม


In [35]:
prompt1 = PromptTemplate(
    template="แปลโจทย์เกี่ยวกับ {question}"
)

chain1 = prompt1 | llm

In [36]:
prompt2 = PromptTemplate(
    template="ทำความเข้าใจโจทย์ที่แปลแล้วต่อไปนี้ {translate}"
)

chain2 = prompt2 | llm

In [37]:
prompt3 = PromptTemplate(
    template="แก้โจทย์ตามที่เข้าใจต่อไปนี้ {understand}"
)

chain3 = prompt3 | llm

In [38]:
prompt4 = PromptTemplate(
    template="อธิบายวิธิีทำจากผลลัทธ์ต่อไปนี้ {solve}"
)

chain4 = prompt4 | llm

In [39]:
parser_chain = StrOutputParser()

In [40]:
full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [41]:
result = full_chain.invoke({"question":"2x + 5 = 15"})

In [42]:
result

'วิธิีทำจากผลลัทธ์ต่อไปนี้ ถูกต้องแล้วครับ\n\n**วิธิีทำจากผลลัทธ์** คือการใช้ผลลัทธิ (เช่น โครงการ, นโยบาย, วิธีการ) ในการตัดสินใจและดำเนินการต่างๆ อย่างมีเหตุผลและมีประสิทธิภาพ โดยการใช้หลักการทางคณิตศาสตร์และสถิติเพื่อหาผลลัพธ์ที่ถูกต้องและเป็นไปตามข้อกำหนดของเหตุการณ์นั้นๆ\n\n**ตัวอย่าง:**\n\n* **การตัดสินใจของรัฐบาล:** รัฐบาลอาจใช้ผลลัทธิการเมืองเพื่อวางแผนการจัดตั้งรัฐบาลที่มีประสิทธิภาพ มีความน่าเชื่อถือ และสามารถตอบสนองความต้องการของประชาชนได้\n* **การจัดการกับปัญหา:** การจัดการกับปัญหาที่ซับซ้อนอาจต้องใช้ผลลัทธิในการวิเคราะห์ปัญหา วิเคราะห์ข้อมูล และหาแนวทางแก้ไขที่เหมาะสม\n* **การพัฒนาผลิตภัณฑ์:** การพัฒนาผลิตภัณฑ์ใหม่อาจต้องใช้ผลลัทธิในการวิเคราะห์ตลาด การทดลองผลิตภัณฑ์ และการปรับปรุงคุณภาพ\n\n**ความสำคัญของวิธิีทำจากผลลัทธ์:**\n\n* **ความแม่นยำและความน่าเชื่อถือ:** ผลลัทธิช่วยให้การตัดสินใจมีความแม่นยำและน่าเชื่อถือ\n* **การปรับตัวและการปรับปรุง:** ผลลัทธิสามารถช่วยให้มีการปรับตัวและปรับปรุงผลลัพธ์ได้อย่างรวดเร็วและมีประสิทธิภาพ\n* **การแก้ปัญหา:** ผลลัทธิสามารถช่วยในการแก้ป

In [43]:
prompt_template = """
[1. Role/Context บอก LLM ว่าควรทำตัวเป็นอะไร]
คุณเป็น AI ที่เชี่ยวชาญในการวิเคราะห์รีวิวภาษาไทย

[2. Task Description บอกงานที่ต้องทำ]
จาก Review ต่อไปนี้ ให้ Extract 3 keywords
ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

[3. Input Data ข้อมูลที่ต้องประมวลผล]
Review: {review}

[4. Output Format บอกรูปแบบของคำตอบ]
ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}

[5. Constraints ข้อจำกัดหรือเงื่อนไข]
- ต้องเป็นคำภาษาไทย
- คั่นด้วย comma
- หากไม่พบให้ตอบ 'ไม่ระบุ'

[6. Examples (optional) ตัวอย่าง input/output]
ตัวอย่าง
Input: "กาแฟอร่อย บรรยากาศดี ราคาแพง"
Output: {"keywords": "กาแฟ, บรรยากาศ, ราคา"}
"""

In [44]:
prompt = """
Extract keywords from this review
{review}
"""

# Pros Simple, fast
# Cons อาจได้ผลไม่ดีกับ Tasks ซับซ้อน

In [45]:
prompt = """
Extract keywords from reviews.

Example 1
Review: "กาแฟอร่อย บริการดี"
Keywords: กาแฟ, บริการ

Example 2
Review: "ราคาแพง แต่คุ้มค่า"
Keywords: ราคา, คุณภาพ

Now extract from this review
Review: {review}
Keywords:
"""

# Pros ได้ผลดีกว่า Zero-shot
# Cons Prompt ใช้ Tokens มากขึ้น

In [46]:
prompt = """
Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON: {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
"""

# Pros ได้ผลดีกับ Reasoning tasks
# Cons ใช้ Tokens มากกว่า ช้ากว่า

In [47]:
keyword_prompt = PromptTemplate(
template = """
จาก Review ต่อไปนี้
Extract 3 keywords ที่เป็นตัวแทนของ Review นี้

Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON โดยมี key คือ keyword เช่น :{{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
""")

In [48]:
sample_reviews = coffee_reviews_df['column0'].head(2)
sample_reviews

,column0
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...


In [49]:
sample_reviews.to_csv('/content/sample_reviews.csv', index=False)

In [50]:
keyword_extract = keyword_prompt | llm | StrOutputParser()

In [51]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 54.4 ms, sys: 7.59 ms, total: 62 ms
Wall time: 6.69 s


In [52]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{
  "keywords": [
    "ร้านอาหารใหญ่",
    "ห้องน้ำ",
    "กาแฟ",
    "น้ำผึ้ง",
    "แพงเวอร์",
    "อย่าสั่งเลย",
    "ลาบไข่ต้ม",
    "ไข่มันคาว",
    "ไม่ประทับใจ",
    "ทอดมันหัวปลีกรอบอร่อย",
    "เบิ้ล",
    "ขนมไทย",
    "ขนมหวาน",
    "ฟรีเล็กน้อย"
  ]
}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
{
  "keywords": [
    "ลาเต้",
    "ร้อน",
    "แก้วนึง",
    "จัดไป",
    "ตึก Software Park",
    "ไรกินแถวนี้",
    "แถวอารีย์",
    "ซดกาแฟ",
    "กาแฟที่นี่",
    "ทานกาแฟ",
    "ทานกาแฟสวย",
    "กาแฟอร่อย",
    "กาแฟราคาไม่แพง",
    "กินบ่อยก็จุกได้",
    "กาแฟจุก",
    "กาแฟดี",
    "กาแฟอร่อย",
    "กาแฟราคาไม่แพง",
    "กินบ่อย"
  ]
}
```



In [53]:
!ollama pull scb10x/typhoon2.1-gemma3-4b

In [54]:
!ollama list

NAME                                  ID              SIZE      MODIFIED           
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    13 seconds ago        
gemma3:270m                           e7d36fb2c3b3    291 MB    About a minute ago    


In [55]:
llm3 = OllamaLLM(model="gemma3:270m")

In [56]:
keyword_extract = keyword_prompt | llm3 | StrOutputParser()

In [57]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 34.1 ms, sys: 3.44 ms, total: 37.5 ms
Wall time: 3.65 s


In [58]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{
  "keywords": [
    "ร้านอาหารใหญ่",
    "ห้องน้ำ",
    "กาแฟ",
    "น้ำผึ้ง",
    "แพง",
    "ไม่ประทับใจ",
    "ราคาสูง",
    "เกินไป",
    "รับไม่ไหว",
    "วุ่นวาย",
    "ขนมหวานฟรี",
    "ขนมไทย",
    "ขนม"
  ]
}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
"keywords": [
  "ลาเต้",
  "รสชาติ",
  "กลิ่น",
  "มวยรุ่นเดียวกะ Starbuck",
  "เมพพพ",
  "ซากอารยธรรมลาเต้"
]
```



In [59]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class ReviewKeywords(BaseModel):
    keywords: str = Field(
        ...,  # Required field
        description="3 keywords separated by comma",
        examples=["กาแฟ, บรรยากาศ, ราคา"]
    )

class KeywordsExtraction(BaseModel):
    extracted_qualities: ReviewKeywords

In [60]:
keywords_parser = JsonOutputParser(pydantic_object=KeywordsExtraction)

In [70]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [71]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 60.5 ms, sys: 8.7 ms, total: 69.2 ms
Wall time: 6.75 s


In [72]:
results[0]['text']['keywords']

['ร้านอาหารใหญ่',
 'ห้องน้ำ',
 'กาแฟ',
 'น้ำผึ้ง',
 'แพง',
 'ไม่แนะนำ',
 'ลดราคา',
 'ฟรี',
 'ขนมไทย',
 'ขนมหวาน',
 'ลาบไข่ต้ม',
 'ไข่อร่อย',
 'เบิ้ล',
 'ทอด',
 'หัวปลีก',
 'อร่อย',
 'ราคา',
 'เกินไป',
 'รับไม่ไหว',
 'กินไม่ไหว',
 'ขนมหวานให้กินฟรี',
 'ขนมไทย',
 'ขนมหวาน']

In [73]:
results[1]['text']['keywords']

['กาแฟ',
 'ร้านวาวี',
 'อารีย์',
 'Starbuck',
 'แฟนๆ Starbuck',
 'กาแฟอร่อย',
 'ราคาไม่แพง',
 'กินบ่อย',
 'จุกได้',
 'ซอยไรจำไม่ได้',
 'แต่อยู่ตรงข้าม Villa',
 'Sunshine',
 'สวย',
 'มวยรุ่นเดียวกะ Starbuck',
 'Cocina',
 'เครื่องดื่ม',
 'รสชาติ',
 'กลิ่น',
 'เมพพพ',
 'เมนู',
 'น้ำส้มกะเค้ก',
 'แจ่ม',
 'ซากอารยธรรมลาเต้เมพ']

In [74]:
llm3 = OllamaLLM(
    model="scb10x/typhoon2.1-gemma3-4b",
    temperature=0.05
)

keyword_extract = keyword_prompt | llm3 | keywords_parser

In [75]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 152 ms, sys: 12.1 ms, total: 164 ms
Wall time: 18 s


In [76]:
results[0]['text']['keywords'], results[1]['text']['keywords']

(['ราคา', 'รสชาติ', 'ขนาด'], ['กาแฟ', 'บรรยากาศ', 'วาวี'])

In [77]:
keyword_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract keywords 3 keywords เท่านั้น ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

    Review: {review}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
    และค่าเป็นข้อความที่ดึงได้คั่นด้วย comma
    เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}}

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [78]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [79]:
inputs = [{"review": r} for r in sample_reviews]

In [80]:
%%time
results = keyword_extract.batch(inputs)

CPU times: user 19.1 ms, sys: 1.51 ms, total: 20.7 ms
Wall time: 2.8 s


In [81]:
results

[{'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
 {'keywords': 'กาแฟ, บรรยากาศ, รสชาติ'}]

In [82]:
sample_reviews_1000 = coffee_reviews_df['column0'].head(1000).tolist()
inputs_1000 = [{"review": r} for r in sample_reviews_1000]

BATCH_SIZE = 8

results_1000 = []

In [83]:
from tqdm import tqdm

In [84]:
for i in tqdm(range(0, len(inputs_1000), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_1000[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_1000.append({"review": inp["review"], "text": out})

Extracting keywords: 100%|██████████| 125/125 [19:34<00:00,  9.39s/it]


In [85]:
for result in results_1000[0:10]:
    print(result['text']['keywords'])

ราคา, บรรยากาศ, ไข่ต้ม
กาแฟ, บรรยากาศ, รสชาติ
โดนัท, รสชาติ, ความหวาน
วาซาบิ, ปลาซาบะ, ข้าวหน้าปลา
โดนัท, Original Glazed, คิว
วังพญาไท, นรสิงห์, บรรยากาศคลาสสิก
อาหารทะเล, ราคา, บรรยากาศ
กาแฟ, เค้ก, พนักงาน
ฮันนี่ โทสท์, ราคา, รสชาติ
คาเฟ่, วานิลลา, คาโบนาร่า


In [86]:
  results_1000[0]

{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
 'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'}}

In [87]:
context_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract context ใน Review ที่เป็นตัวแทนที่บรรยายคำว่า {keyword} ได้ดีที่สุด

    Review: {review}

    ตอบผลลัพธ์ในรูปแบบ JSON โดยมีคีย์เป็น 'context'
    และค่าเป็นข้อความที่ดึงได้จาก Review

    ตัวอย่าง
    keyword คือ "กาแฟ"
    context: "กาแฟที่นี่อร่อยมาก หอมกลิ่นกาแฟคั่วสด"

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [88]:
sentiment_prompt = PromptTemplate(
    template="""จากข้อความต่อไปนี้ ให้วิเคราะห์ว่าเป็น sentiment แบบไหน
    เลือกจากตัวเลือกเหล่านี้เท่านั้น: "positive", "negative", "neutral"

    ข้อความ: {message}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'sentiment'
    และค่าเป็นหนึ่งใน list ต่อไปนี้ [positive, negative, neutral]

    คำแนะนำ
    - positive คือ ข้อความที่แสดงความพอใจ ชอบ ดี
    - negative คือ ข้อความที่แสดงความไม่พอใจ ไม่ชอบ แย่
    - neutral คือ ข้อความที่ไม่แสดงความรู้สึกชัดเจน เป็นกลาง

    # ตัวอย่าง
    # ข้อความ "อร่อยมาก" → {{"sentiment": "positive"}}
    # ข้อความ "แพงไป" → {{"sentiment": "negative"}}
    # ข้อความ "ปกติ" → {{"sentiment": "neutral"}}
    """
)


In [89]:
class ContextKeyword(BaseModel):
    context: str = Field(..., description="context that best represent this keyword")

class ContexExtraction(BaseModel):
    extracted: ContextKeyword

context_parser = JsonOutputParser(pydantic_object=ContexExtraction)

In [90]:
context_extract = context_prompt | llm3 | context_parser

In [91]:
class Sentiment(BaseModel):
    sentiment: str = Field(..., description="sentiment of review")

class SentimentExtraction(BaseModel):
    extracted: Sentiment


sentiment_parser = JsonOutputParser(pydantic_object=SentimentExtraction)

In [92]:
sentiment_extract = sentiment_prompt | llm3 | sentiment_parser

In [93]:
keywords = results_1000[0]['text']['keywords']


In [94]:
context_inputs = []

for review_data in tqdm(results_1000, desc="Preparing context inputs"):
    review = review_data['review']
    keywords = review_data['text']['keywords']
    for keyword in keywords:
        context_inputs.append({"review": review, "keyword": keyword.strip()})

Preparing context inputs: 100%|██████████| 1000/1000 [00:00<00:00, 66987.75it/s]


In [95]:
len(context_inputs)

24808

In [96]:
context_results = []
for i in tqdm(range(len(results_1000)), desc="Extracting contexts"):
   result = context_extract.batch(context_inputs[i*3:(i*3)+3])
   contexts_list = [d['context'] for d in result]
   results_1000[i]['contexts'] = contexts_list

Extracting contexts: 100%|██████████| 1000/1000 [1:25:33<00:00,  5.13s/it]


In [97]:
results_1000[:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ร้านอาหารใหญ่มากกกกกกก',
   'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี',
   'ห้องทานอาหารอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสว

In [98]:
sentiment_inputs = []

for review_data in tqdm(results_1000, desc="Preparing sentiment inputs"):
    contexts = review_data['contexts']
    for context in contexts:
        sentiment_inputs.append({"message": context})

Preparing sentiment inputs: 100%|██████████| 1000/1000 [00:00<00:00, 377355.29it/s]


In [99]:
len(sentiment_inputs)

3000

In [100]:
sentiment_inputs[:6]

[{'message': 'ร้านอาหารใหญ่มากกกกกกก'},
 {'message': 'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี'},
 {'message': 'ห้องทานอาหารอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย'},
 {'message': 'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี'},
 {'message': 'ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว'},
 {'message': 'ทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล'}]

In [101]:
for i in tqdm(range(len(results_1000)), desc="Extracting sentiment"):
   result = sentiment_extract.batch(sentiment_inputs[i*3:(i*3)+3])
   sentiment_list = [d['sentiment'] for d in result]
   results_1000[i]['sentiment'] = sentiment_list

Extracting sentiment: 100%|██████████| 1000/1000 [42:19<00:00,  2.54s/it]


In [102]:
results_1000[0:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ร้านอาหารใหญ่มากกกกกกก',
   'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี',
   'ห้องทานอาหารอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย'],
  'sentiment': ['neutral', 'negative', 'negative']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรย

In [103]:
import pandas as pd

df = pd.DataFrame([
    {
        'review': r['review'],
        **{f'keyword{i+1}': k.strip() for i, k in enumerate(r['text']['keywords'])},
        **{f'context{i+1}_keyword': c for i, c in enumerate(r['contexts'])},
        **{f'sentiment{i+1}': s for i, s in enumerate(r['sentiment'])}
    }
    for r in results_1000
])

In [104]:
df.to_csv('results.csv', index=False, encoding='utf-8-sig')